<a href="https://colab.research.google.com/github/sabesansathananthan/tamil-song-corpus/blob/master/Web_scraping_tamilpaa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

This notebook is meant to serve as a small example project of how to get data from the web using Python. Here we will scrape the data from the web, parse the results using regular expressions, and visualize the data. Here we use TamilPaa website to scrap the tamil songs.

# Web Scraping Using Requests and Beautiful Soup (bs4)

In [2]:
# requests for fetching html of website
import requests

# Make the request to a url
r = requests.get('https://www.tamilpaa.com/tamil-movies-list')

import numpy as np
import pandas as pd
# Create soup from content of request
c = r.content
import json
from bs4 import BeautifulSoup

soup = BeautifulSoup(c)




In [3]:
wi=['0','1','2','3','4','5','6','7','8','9','a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


We can find an element on the page by inspecting the page (right click and hit inspect element). We then use a series of HTML selectors to find the appropriate tags which contain the content we are interested in. The next code block finds the main text of the entire article. We will then further subset the text to the relevant table and save it as a text object. 

In [5]:
data = []


In [10]:
url= 'http://www.tamilcollections.com/lyrics-movies.aspx?index='
data=[]
for i in wi:
  urli = url+i
  r = requests.get(urli)
  c = r.content
  soup = BeautifulSoup(c)
  detail_content = soup.find('table', attrs = {'class': 'table table-hover'})
  try:
    tablecontent =detail_content.find_all('tr')
  except:
    print(i,0)
    continue
  for td in tablecontent[1:]:
    song = {}
    ass = td.find_all('a')
    song['movie_name'] = ass[0].get_text()
    song['song_name'] = ass[1].get_text()
    song['song_url'] = ass[1].get('href')
    data.append(song)



0 0
2 0
5 0
6 0
8 0
9 0
x 0


In [11]:
data

[{'movie_name': '12 B (2001)',
  'song_name': 'Jothi Nerainjava',
  'song_url': 'http://www.tamilcollections.com/lyrics/jothi-nerainjava/612/'},
 {'movie_name': '12 B (2001)',
  'song_name': 'Love Pannu',
  'song_url': 'http://www.tamilcollections.com/lyrics/love-pannu/613/'},
 {'movie_name': '12 B (2001)',
  'song_name': 'Mutham Mutham',
  'song_url': 'http://www.tamilcollections.com/lyrics/mutham-mutham/614/'},
 {'movie_name': '12 B (2001)',
  'song_name': 'Poove Vai Pesum',
  'song_url': 'http://www.tamilcollections.com/lyrics/poove-vai-pesum/615/'},
 {'movie_name': '12 B (2001)',
  'song_name': 'Sariya Thavara',
  'song_url': 'http://www.tamilcollections.com/lyrics/sariya-thavara/616/'},
 {'movie_name': '123 (2002)',
  'song_name': 'Adada Nadandhu Vara',
  'song_url': 'http://www.tamilcollections.com/lyrics/adada-nadandhu-vara/2488/'},
 {'movie_name': '16 Vayathinile',
  'song_name': 'Aattu Kutti Muttai Ittu',
  'song_url': 'http://www.tamilcollections.com/lyrics/aattu-kutti-muttai

In [17]:
def viewMovie(movie):
  r = requests.get(movie['movie_url'])
  c = r.content
  soup = BeautifulSoup(c)
  detail_content = soup.find('table', attrs = {'class': 'standard'})
  tablecontent =detail_content.find_all('td')
  movie['movie_image']= tablecontent[0].find('img').get('src')
  movie['movie_name_tamil']= tablecontent[2].get_text()
  movie['movie_name_eng']=tablecontent[1].get_text()
  movie['movie_song'] = []
  song_content = soup.find('div', attrs = {'class': 'tab-content clearfix', 'id': 'tab_1'})
  songcontent =song_content.find_all('li')
  
  for j in songcontent:
    song ={}
    song['song_name']= j.find('a').get_text()
    song['song_url']= j.find('a').get('href')
    movie['movie_song'].append(song)
  return movie

In [18]:
def viewSong(song):
  r = requests.get(song['song_url'])
  c = r.content
  soup = BeautifulSoup(c)
  detail_content = soup.find('table', attrs = {'class': 'standard mb-10px'})
  tablecontent =detail_content.find_all('td')
  song['song_music']= tablecontent[1].get_text()
  song['song_lyrics']= tablecontent[3].get_text()
  song['song_singers']= tablecontent[4].get_text()
  song_content = soup.find('div', attrs = {'class': 'info-box white-bg'})

  song['song_fulllyrics']= song_content.get_text()
  return song
    

In [23]:
for imo in range(len(data)):
  try:
    mo = data[imo]
    mo = viewMovie(mo)
    for iso in range(len(mo['movie_song'])):
      mo['movie_song'][iso] = viewSong( mo['movie_song'][iso])
    data[imo] = mo
    print(imo/10.0,"%")
    if (imo%7==0):
      datajson = {}
      datajson['movies']=data
      with open('data.json', 'w') as fp:
        json.dump(datajson, fp)
      df=pd.DataFrame.from_dict(datajson)
      df.to_csv('tamilpaa_data.txt', index=False)
      !cp data.json "/content/drive/My Drive/Colab Notebooks/Data"
      !cp tamilpaa_data.txt "/content/drive/My Drive/Colab Notebooks/Data"!cp data.json "/content/drive/My Drive/Colab Notebooks/Data"
      print('Saved!!!',imo)
  except:
    print("Error on ",imo)




Error on  0
Error on  1
Error on  2
Error on  3
Error on  4
Error on  5
Error on  6
Error on  7
Error on  8
Error on  9
Error on  10
Error on  11
Error on  12
Error on  13
Error on  14
Error on  15
Error on  16
Error on  17
Error on  18
Error on  19
Error on  20
Error on  21
Error on  22
Error on  23
Error on  24
Error on  25
Error on  26
Error on  27
Error on  28
Error on  29
Error on  30
Error on  31
Error on  32
Error on  33
Error on  34
Error on  35
Error on  36
Error on  37
Error on  38
Error on  39
Error on  40
Error on  41
Error on  42
Error on  43
Error on  44
Error on  45
Error on  46
Error on  47
Error on  48
Error on  49
Error on  50
Error on  51
Error on  52
Error on  53
Error on  54
Error on  55
Error on  56
Error on  57
Error on  58
Error on  59
Error on  60
Error on  61
Error on  62
Error on  63
Error on  64
Error on  65
Error on  66
Error on  67
Error on  68
Error on  69
Error on  70
Error on  71
Error on  72
Error on  73
Error on  74
Error on  75
Error on  76
Error on 

In [20]:
datajson = {}
datajson['movies']=data
with open('data.json', 'w') as fp:
  json.dump(datajson, fp)
!cp data.json "/content/drive/My Drive/Colab Notebooks/Data"

In [21]:
df=pd.DataFrame.from_dict(datajson)
df.to_csv('tamilpaa_data.txt', index=False)